# Class 21: Searching the NCBI databases
**April 9, 2019**

## Different ways of processing Entrez results

Every time we download data through the Entrez module, we can interact with the results in different ways. First, we can just use the handle we obtain as an ordinary file handle and just store or process the raw data provided by Entrez. Second, we can process the data with an appropriate, existing Biopython module. The latter will generally be preferable if an appropriate module exists. However, the various choices that are available may make things confusing.

As an example, we will again download the genbank record with the ID "KT220438", containing an influenza HA protein. We will consider four different ways of looking at the data. First, we use `retmode="text"` in `Entrez.efetch()` and just download the raw data and print it. We get a regular genbank file as output:

In [5]:
from Bio import Entrez, SeqIO
Entrez.email = "akshayvaranasi@austin.utexas.edu" # put your email here

# Download sequence record for genbank id KT220438 (HA from influenza A)
# Using text mode
handle = Entrez.efetch(db="nucleotide", id="KT220438", rettype="gb", retmode="text")
record = handle.read() # read file directly
print(record)
handle.close()

LOCUS       KT220438                1701 bp    cRNA    linear   VRL 20-JUL-2015
DEFINITION  Influenza A virus (A/NewJersey/NHRC_93219/2015(H3N2)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KT220438
VERSION     KT220438.1
KEYWORDS    .
SOURCE      Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))
  ORGANISM  Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1701)
  AUTHORS   Sitz,C.R., Thammavong,H.L., Balansay-Ames,M.S., Hawksworth,A.W.,
            Myers,C.A. and Brice,G.T.
  TITLE     GEISS Influenza Surveillance Response Program
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1701)
  AUTHORS   Sitz,C.R., Thammavong,H.L., Balansay-Ames,M.S., Hawksworth,A.W.,
            Myers,C.A. and Brice,G.T.
  TITLE     Direct Submission
  JOURNAL   Submitted (29-JUN-2015) Operational Infectious Diseases, Naval
   

We can also, as we have done before, process this file using the `SeqIO` module:

In [6]:
# Download sequence record for genbank id KT220438 (HA from influenza A)
# Using text mode
handle = Entrez.efetch(db="nucleotide", id="KT220438", rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank") # parse with SeqIO
print(record)
handle.close()

ID: KT220438.1
Name: KT220438
Description: Influenza A virus (A/NewJersey/NHRC_93219/2015(H3N2)) segment 4 hemagglutinin (HA) gene, complete cds
Number of features: 5
/molecule_type=cRNA
/topology=linear
/data_file_division=VRL
/date=20-JUL-2015
/accessions=['KT220438']
/sequence_version=1
/keywords=['']
/source=Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))
/organism=Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))
/taxonomy=['Viruses', 'ssRNA viruses', 'ssRNA negative-strand viruses', 'Orthomyxoviridae', 'Influenzavirus A']
/references=[Reference(title='GEISS Influenza Surveillance Response Program', ...), Reference(title='Direct Submission', ...)]
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Sequencing Technology', 'Sanger dideoxy sequencing')]))])
Seq('ATGAAGACTATCATTGCTTTGAGCTACATTCTATGTCTGGTTTTCGCTCAAAAA...TGA', IUPACAmbiguousDNA())


In addition to text mode, we can also download the data in XML mode. XML is a structured data format that allows for easy machine-processing of complex data files. If we just print the raw data, though, it doesn't look appealing:

In [7]:
# Download sequence record for genbank id KT220438 (HA from influenza A)
# Using XML mode
handle = Entrez.efetch(db="nucleotide", id="KT220438", rettype="gb", retmode="xml")
record = handle.read() # read file directly
print(record)
handle.close()

<?xml version="1.0" encoding="UTF-8"  ?>
<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">
<GBSet>
  <GBSeq>

    <GBSeq_locus>KT220438</GBSeq_locus>
    <GBSeq_length>1701</GBSeq_length>
    <GBSeq_strandedness>single</GBSeq_strandedness>
    <GBSeq_moltype>cRNA</GBSeq_moltype>
    <GBSeq_topology>linear</GBSeq_topology>
    <GBSeq_division>VRL</GBSeq_division>
    <GBSeq_update-date>20-JUL-2015</GBSeq_update-date>
    <GBSeq_create-date>20-JUL-2015</GBSeq_create-date>
    <GBSeq_definition>Influenza A virus (A/NewJersey/NHRC_93219/2015(H3N2)) segment 4 hemagglutinin (HA) gene, complete cds</GBSeq_definition>
    <GBSeq_primary-accession>KT220438</GBSeq_primary-accession>
    <GBSeq_accession-version>KT220438.1</GBSeq_accession-version>
    <GBSeq_other-seqids>
      <GBSeqid>gb|KT220438.1|</GBSeqid>
      <GBSeqid>gi|887493048</GBSeqid>
    </GBSeq_other-seqids>
    <GBSeq_source>Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))</

The advantage of XML mode is that there is the generic `Entrez.parse()` function that can parse XML files returned from `Entrez.efetch()`. Also, some modules in Biopython cannot work with files obtained in text mode, they can only work on files obtained in XML mode. The documentation will generally tell you for each module what kind of input it expects.

Reading the above example with `Entrez.parse()` gives us the following:  

In [8]:
# Download sequence record for genbank id KT220438 (HA from influenza A)
handle = Entrez.efetch(db="nucleotide", id="KT220438", rettype="gb", retmode="xml")
parsed = Entrez.parse(handle)
record = list(parsed)[0] # We need to convert the parsed contents into a list. Here we want just the 0th element.
handle.close()
print(record)

DictElement({'GBSeq_locus': 'KT220438', 'GBSeq_length': '1701', 'GBSeq_strandedness': 'single', 'GBSeq_moltype': 'cRNA', 'GBSeq_topology': 'linear', 'GBSeq_division': 'VRL', 'GBSeq_update-date': '20-JUL-2015', 'GBSeq_create-date': '20-JUL-2015', 'GBSeq_definition': 'Influenza A virus (A/NewJersey/NHRC_93219/2015(H3N2)) segment 4 hemagglutinin (HA) gene, complete cds', 'GBSeq_primary-accession': 'KT220438', 'GBSeq_accession-version': 'KT220438.1', 'GBSeq_other-seqids': ['gb|KT220438.1|', 'gi|887493048'], 'GBSeq_source': 'Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))', 'GBSeq_organism': 'Influenza A virus (A/New Jersey/NHRC_93219/2015(H3N2))', 'GBSeq_taxonomy': 'Viruses; ssRNA viruses; ssRNA negative-strand viruses; Orthomyxoviridae; Influenzavirus A', 'GBSeq_references': [DictElement({'GBReference_reference': '1', 'GBReference_position': '1..1701', 'GBReference_authors': ['Sitz,C.R.', 'Thammavong,H.L.', 'Balansay-Ames,M.S.', 'Hawksworth,A.W.', 'Myers,C.A.', 'Brice,G.T.'], 'GBRe

While this output may not seem useful, we now just have a set of nested dictionaries that we can interrogate using standard Python techniques:

In [9]:
print(list(record.keys())) # print out all the keys in the dictionary

['GBSeq_locus', 'GBSeq_length', 'GBSeq_strandedness', 'GBSeq_moltype', 'GBSeq_topology', 'GBSeq_division', 'GBSeq_update-date', 'GBSeq_create-date', 'GBSeq_definition', 'GBSeq_primary-accession', 'GBSeq_accession-version', 'GBSeq_other-seqids', 'GBSeq_source', 'GBSeq_organism', 'GBSeq_taxonomy', 'GBSeq_references', 'GBSeq_comment', 'GBSeq_feature-table', 'GBSeq_sequence']


In [10]:
print(record['GBSeq_sequence']) # print out the sequence

atgaagactatcattgctttgagctacattctatgtctggttttcgctcaaaaaattcctggaaatgacaatagcacggcaacgctgtgccttgggcaccatgcagtaccaaacggaacgatagtgaaaacaatcacaaatgaccgaattgaagttactaatgctactgagctggttcagaattcctcaataggtgaaatatgcgacagtcctcatcagatccttgatggagaaaactgcacactaatagatgctctattgggagaccctcagtgtgatggctttcaaaataagaaatgggacctttttgttgaacgaagcaaagcctacagcaactgctacccttatgatgtgccggattatgcctcccttaggtcactagttgcctcatccggcacactggagtttaacaatgaaagcttcaattggactggagtcactcaaaacggaacaagttctgcttgcataaggagatctagtagtagtttctttagtagattaaattggttgacccacttaaactacacatacccagcattgaacgtgactatgccaaacaatgaacaatttgacaaattgtacatttggggggttcaccacccgggtacggacaaggaccaaatcttcctgtatgctcaatcatcaggaagaatcacagtatctaccaaaagaagccaacaagctgtaatcccaaatatcggatctagacccagaataagggatatccctagcagaataagcatctattggacaatagtaaaaccgggagacatacttttgattaacagcacagggaatctaattgctcctaggggttacttcaaaatacgaagtgggaaaagctcaataatgagatcagatgcacccattggcaaatgcaagtctgaatgcatcactccaaatggaagcattcccaatgacaaaccattccaaaatgtaaacaggatcacatacggggcctgtcccagatatgttaagcatagcactctaaaattggcaa

In [11]:
features = record['GBSeq_feature-table'] # extract all the features
for feature in features: # loop over features and print feature key and feature location
    print(feature['GBFeature_key'] + ": " + feature['GBFeature_location'])

source: 1..1701
gene: 1..1701
CDS: 1..1701
mat_peptide: 49..1035
mat_peptide: 1036..1698


## Running search queries through Entrez

So far we have only downloaded specific records from Entrez. In addition to just downloading records, however, we can also run searches directly from python. Any query that you can do on the Entrez website (https://www.ncbi.nlm.nih.gov/) can also be executed directly from python. This allows you to find a large number of records all at once and process them in an automated fashion.

For example, below we will see how to automatically run and retrieve the results for the following search term: "influenza a virus texas h1n1 hemagglutinin complete cds". A direct link to the search results on the Entrez website is here:    
https://www.ncbi.nlm.nih.gov/nuccore/?term=influenza+a+virus+texas+h1n1+hemagglutinin+complete+cds

(Note that in the following Python code, we limit the number of search hits returned to the first 10.)

In [14]:
# let's do a search for influenza H1N1 viruses from Texas
handle = Entrez.esearch(db="nucleotide",  # database to search
                        term="influenza a virus texas h1n1 hemagglutinin complete cds",  # search term
                        retmax=10  # number of results that are returned
                        )
record = Entrez.read(handle)
handle.close()

gi_list = record["IdList"] # list of genbank identifiers found
print(gi_list)

['1604349740', '1604349721', '1597259311', '1597259287', '1597258925', '1597258900', '1591440116', '1591440105', '1591440078', '1590855198']


Note that even though NCBI is [phasing out sequence GI numbers,](https://www.ncbi.nlm.nih.gov/news/03-02-2016-phase-out-of-GI-numbers/) for now the `esearch()` function still returns GI numbers (numerical sequence identifiers without version information).

We can download all the genbank records in the list of identifiers using the `Entrez.efetch()` function. This function provides us with a handle that needs to be processed with `SeqIO.parse()`. (We used `SeqIO.read()` previously, which reads a single record. `SeqIO.parse()` reads multiple records. See [here](http://biopython.org/wiki/SeqIO) for details.) 

In [15]:
handle = Entrez.efetch(db="nucleotide", id=gi_list, rettype="gb", retmode="text")
records = SeqIO.parse(handle, "genbank")

for record in records:
    print(record.description)
    
handle.close() # important, close the handle only after you have iterated over the records. Otherwise you will get an error!

Influenza A virus (A/Texas/16/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/15/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/12/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/07/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/06/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/05/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/157/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/143/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/02/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/148/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds


As another example, let's search the "pubmed" database (database of scientific publications) for papers from 2015 written by "Wilke CO". The exact search term we need to use is the following: "Wilke CO[Author] AND 2015[Date - Publication]"

You can [click here](http://www.ncbi.nlm.nih.gov/pubmed/?term=Wilke+CO%5BAuthor%5D+AND+2015%5BDate+-+Publication%5D) to see the result from that search online.

In [16]:
handle = Entrez.esearch(db="pubmed",  # database to search
                        term="Wilke CO[Author] AND 2015[Date - Publication]",  # search term
                        retmax=10  # number of results that are returned
                        )
record = Entrez.read(handle)
handle.close()

# search returns PubMed IDs (pmids)
pmid_list = record["IdList"]
print(pmid_list)

['26770819', '26468068', '26430238', '26397960', '26355089', '26275208', '26020774', '25999509', '25787027', '25737813']


Just like with genes and genomes, we can download the records corresponding to these identifiers. They are references. We'll print the author(s), title, and reference (source).

In [17]:
from Bio import Medline
handle = Entrez.efetch(db="pubmed", id=pmid_list, rettype="medline", retmode="text")
records = Medline.parse(handle)
for record in records:
    print(record['AU']) # author list
    print(record['TI']) # title
    print(record['SO']) # source (reference)
    print()
handle.close()

['Meyer AG', 'Spielman SJ', 'Bedford T', 'Wilke CO']
Time dependence of evolutionary metrics during the 2009 pandemic influenza virus outbreak.
Virus Evol. 2015 Jan;1(1). doi: 10.1093/ve/vev006. Epub 2015 Jan 1.

['Meyer AG', 'Wilke CO']
The utility of protein structure as a predictor of site-wise dN/dS varies widely among HIV-1 proteins.
J R Soc Interface. 2015 Oct 6;12(111):20150579. doi: 10.1098/rsif.2015.0579.

['Wilke CO']
Evolutionary paths of least resistance.
Proc Natl Acad Sci U S A. 2015 Oct 13;112(41):12553-4. doi: 10.1073/pnas.1517390112. Epub 2015 Oct 1.

['Spielman SJ', 'Wilke CO']
Pyvolve: A Flexible Python Module for Simulating Sequences along Phylogenies.
PLoS One. 2015 Sep 23;10(9):e0139047. doi: 10.1371/journal.pone.0139047. eCollection 2015.

['Kerr SA', 'Jackson EL', 'Lungu OI', 'Meyer AG', 'Demogines A', 'Ellington AD', 'Georgiou G', 'Wilke CO', 'Sawyer SL']
Computational and Functional Analysis of the Virus-Receptor Interface Reveals Host Range Trade-Offs in New 

## Problems

**Problem 1**

Use the following code to download the genbank record `KT220438` in XML and parse it with the `Entrez.parse()` function:

In [19]:
# Download sequence record for genbank id KT220438 (HA from influenza A)
handle = Entrez.efetch(db="nucleotide", id="KT220438", rettype="gb", retmode="xml")
parsed = Entrez.parse(handle)
record = list(parsed)[0] # Convert the parsed contents into a list and take element 0.
handle.close()

Then:

(a) Print out the value for the key `GBSeq_definition`.

(b) Find the `CDS` feature and print out all its qualifiers. Note that qualifiers are provided under the keyword `GBFeature_quals`.

In [20]:
# Problem 1a

# Your code goes here.
print(record['GBSeq_definition'])

Influenza A virus (A/NewJersey/NHRC_93219/2015(H3N2)) segment 4 hemagglutinin (HA) gene, complete cds


In [23]:
# Problem 1b

# Your code goes here.
features = record['GBSeq_feature-table'] # extract all the features
for feature in features: # loop over features and find CDS feature
    if feature['GBFeature_key']=='CDS':
        CDS_feature = feature
        break
qualifiers = CDS_feature['GBFeature_quals']
for q in qualifiers:
    print(q['GBQualifier_name'] + ": " + q['GBQualifier_value'])


gene: HA
function: receptor binding and fusion protein
codon_start: 1
transl_table: 1
product: hemagglutinin
protein_id: AKQ43545.1
translation: MKTIIALSYILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGEICDSPHQILDGENCTLIDALLGDPQCDGFQNKKWDLFVERSKAYSNCYPYDVPDYASLRSLVASSGTLEFNNESFNWTGVTQNGTSSACIRRSSSSFFSRLNWLTHLNYTYPALNVTMPNNEQFDKLYIWGVHHPGTDKDQIFLYAQSSGRITVSTKRSQQAVIPNIGSRPRIRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKHSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQINGKLNRLIGKTNEKFHQIEKEFSEVEGRIQDLEKYVEDTKIDLWSYNAELLVALENQHTXDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNGTYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISXAISCFLLCVALLGFIMWACQKGNIRCNICI


**Problem 2:**

(a) Use an Entrez esearch query of the pubmed database to find out how many publications "Spielman SJ" wrote in 2015.

(b) From the results of part (a), compile a combined list of all the co-authors of "Spielman SJ" in 2015.

In [25]:
# Problem 2a

# Your code goes here.
# Problem 2a
handle = Entrez.esearch(db="pubmed", # database to search
                        term="Spielman SJ[Author] AND 2015[Date - Publication]"
, # search term
                        retmax=10 # number of results that are returned
                       )
record = Entrez.read(handle)
handle.close()

# search returns PubMed IDs (pmids)
pmid_list = record["IdList"]
print("Publications found:", pmid_list)
print("Number of publications:", len(pmid_list))
record

Publications found: ['26770819', '26397960', '25737813', '25576365']
Number of publications: 4


DictElement({'Count': '4', 'RetMax': '4', 'RetStart': '0', 'IdList': ['26770819', '26397960', '25737813', '25576365'], 'TranslationSet': [], 'TranslationStack': [DictElement({'Term': 'Spielman SJ[Author]', 'Field': 'Author', 'Count': '19', 'Explode': 'N'}, attributes={}), DictElement({'Term': '2015[Date - Publication]', 'Field': 'Date - Publication', 'Count': '1257156', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': 'Spielman SJ[Author] AND 2015[Date - Publication]'}, attributes={})

In [27]:
# Problem 2b

# Your code goes here.
from Bio import Medline
handle = Entrez.efetch(db="pubmed", id=pmid_list, rettype="medline", retmode="text")
records = Medline.parse(handle)
coauthors = [] # start with empty list of coauthors
for record in records:
    au_list = record['AU']
    for author in au_list:
        if author != "Spielman SJ" and author not in coauthors:
            coauthors.append(author)
handle.close()
print('Co-authors of "Spielman SJ" in 2015:')
for author in coauthors:
    print(" ", author)

Co-authors of "Spielman SJ" in 2015:
  Meyer AG
  Bedford T
  Wilke CO
  Kumar K


## If this was easy

**Problem 3:**

For larger searches, NCBI wants you to use the WebEnv method to download all your search results. This is explained in the Biopython tutorial [here.](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec:entrez-webenv) Rewrite the influenza search from the section "Running search queries on through Entrez" in such a way that it uses the WebEnv method. For this downloading method, it makes sense to write all the results into a file and then read the results back in.

In [28]:
# Problem 3

# Your code goes here.
handle = Entrez.esearch(db="nucleotide", # database to search
                        term="influenza a virus texas h1n1 hemagglutinin complete cds", # search term
                        usehistory="y" # this switches on the WebEnv method
)
results = Entrez.read(handle)
handle.close()
# Because we ran the search with usehistory="y", the results now contain two additional
# pieces of information, the WebEnv session cookie and the QueryKey:
webenv = results["WebEnv"]
query_key = results["QueryKey"]
# We also get the number of search results:
count = int(results["Count"])
# We now download the results and store them in a local file
# Downloading happens in batches, let's download 20 results at a time:
batch_size = 20
out_handle = open("influenza_HA.gb", "w")
for start in range(0, count, batch_size):
    end = min(count, start+batch_size)
    print("Downloading records %i through %i" % (start+1, end))
    fetch_handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text",
                                 retstart=start, retmax=batch_size,
                                 webenv=webenv, query_key=query_key)
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
out_handle.close()

In [29]:
in_handle = open("influenza_HA.gb", "r")
records = SeqIO.parse(in_handle, "genbank")
for record in records:
    print(record.description)
in_handle.close()

Influenza A virus (A/Texas/16/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/15/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/12/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/07/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/06/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/05/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/157/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/143/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/02/2019(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/148/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/147/2018(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influe

Influenza A virus (A/Texas/23/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/23/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/29/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/18/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/05/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/04/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/47/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/36/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/30/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/47/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza A virus (A/Texas/22/2012(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Influenza 

Influenza A virus (A/Texas/46221665/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46214103/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46201823/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46193632/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/JMS378/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/JMS377/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/JMS376/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/JMS375/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46193311/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46192760/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/461917783/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46182018/2009(H1N1)) segment 4, complete sequence
Influenza A virus (A/Texas/46181292/2009(H1N1)) segment 4, complete sequence
Influe